In [1]:
import pandas as pd

In [7]:
filepath = '/Volumes/backup_128G/z_repository/Yumin_data/玉敏_俄羅斯課本的研究/華語八千詞(內含注音字型檔)'
file_dic = '華語八千詞表20180419_modified.xlsx'
to_dic = 'Chinese_8000W_20190515_v1.xlsx'

read_dic = '{0}/{1}'.format(filepath, file_dic)
write_dic = '{0}/{1}'.format(filepath, to_dic)

In [16]:
def wordRow(word, level):
    return {0:word, 1:level}

In [17]:
def Df2WordList(INdf, level):
    wordList = []
    for idx in range(0, len(INdf)):
        row = INdf.loc[idx]
        word = str(row['詞彙'])
    
        wordList.append(wordRow(word, level))
        
    return wordList

In [21]:
def runParsing(wordList):
    newWordList = []
    for theWord in wordList:
        word = theWord[0]
        level = theWord[1]

        if '/' in word:
            wordArr = word.split('/')
            for sword in wordArr:
                if sword != '':
                    newWordList.append(wordRow(sword, level))
            continue

        if '(' in word:
            wordArr = word.split('(')

            for widx in range(0, len(wordArr)):
                sword = wordArr[widx].replace(')', '').replace(' ', '')
                if widx == 0:
                    newWordList.append(wordRow(sword, level))
                elif sword.isalpha() == True:
                    oriWord = wordArr[0][0:len(wordArr[0])-1]
                    if sword.startswith(oriWord):
                        newWordList.append(wordRow(sword, level))
                    else:
                        completeWord = '{0}{1}'.format(wordArr[0], sword)
                        newWordList.append(wordRow(completeWord, level))

            continue

        newWordList.append(wordRow(word, level))
    
    return newWordList

In [25]:
sheet2Level = {'準備級一級':1,'準備級二級':2,'入門級':3,'基礎級':4,'進階級':5,'高階級':6,'流利級':7}
# sheet2Level = {'準備級一級':1}
with pd.ExcelFile(read_dic) as reader:
    with pd.ExcelWriter(write_dic) as writer:
        # read sheet by sheet
        for sheet in reader.sheet_names:
            if sheet not in sheet2Level:
                continue

            print(sheet)
            dicDf = pd.read_excel(reader, sheet)
#             dicDf = dicDf.fillna(0)

            level = sheet2Level[sheet]
            wordList = Df2WordList(dicDf, level)
#             print(wordList)

            newWordList = runParsing(wordList)
            newWordList = runParsing(newWordList)
#             print(newWordList)

            outDf = pd.DataFrame(newWordList)
            outDf[1] = pd.to_numeric(outDf[1])
            outDf.to_excel(writer, sheet, index=False, header=False)
            writer.save()


準備級一級
準備級二級
入門級
基礎級
進階級
高階級
流利級
